In [ ]:
import tensorflow as tf
import glob
import random
import numpy as np

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
all_image_path = glob.glob("datasets/2_class/*/*.jpg")
random.shuffle(all_image_path)

In [ ]:
label_to_index = {
                    "airplane": 0,
                    "lake": 1
                 }
index_to_label = dict((g, k) for k, g in label_to_index.items())

all_label = [img.split("/")[2] for img in all_image_path]
all_index = [label_to_index.get(label) for label in all_label]

def load_image(path):
    img = tf.io.read_file(path)
    img_tensor = tf.image.decode_jpeg(img, channels=3)
    img_tensor = tf.image.resize(img_tensor, [256, 256])
    img_tensor = tf.cast(img_tensor, tf.float32)
    img_tensor /= 255
    
    return img_tensor

In [ ]:
img_ds = tf.data.Dataset.from_tensor_slices(all_image_path)
img_ds = img_ds.map(load_image)
label_ds = tf.data.Dataset.from_tensor_slices(all_index)
img_label_ds = tf.data.Dataset.zip((img_ds, label_ds))

img_count = len(all_image_path)
test_count = int(img_count * 0.2)
train_count = img_count - test_count

train_ds = img_label_ds.skip(test_count)
test_ds = img_label_ds.take(test_count)

batch_size = 16
train_ds = train_ds.repeat().shuffle(100).batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation="relu"))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same"))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1024, activation="relu"))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.BinaryCrossentropy(), metrics=["acc"])
# steps_per_epoch = train_count / batch_size
# val_steps = test_count / batch_size
steps_per_epoch = 1
val_steps = 1

In [ ]:
history = model.fit(train_ds, epochs=1, 
                    steps_per_epoch=steps_per_epoch, 
                    validation_data=test_ds, 
                    validation_steps=val_steps)